SMT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SMT version.

Necessary libraries:

In [ ]:
!pip install z3-solver

Necessary imports:

In [20]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [21]:
# open the file in Instances folder
f = open("inst07.dat", "r")
# the first line is the number of couriers
m = int(f.readline())
# the second line is the number of items
n = int(f.readline())
# the third line is the load size of each courier
load_size = [int(x) for x in f.readline().split()]
# the fourth line is the size of each item
item_size = [int(x) for x in f.readline().split()]
# the rest is the distance matrix
distance = []
for i in range(n+1):
    distance.append([int(x) for x in f.readline().split()])
# close the file
f.close()
print("couriers:", m)
print("items:", n)
print("load_size:", load_size)
print("item_size:", item_size)
# output the distance matrix as a numpy array
distance = np.array(distance)
print("distance:\n", distance)

couriers: 6
items: 17
load_size: [190, 185, 185, 190, 195, 185]
item_size: [11, 11, 23, 16, 2, 1, 24, 14, 20, 10, 11, 22, 2, 12, 9, 21, 10]
distance:
 [[  0  20  19  28  58  48  45  32  90  61  71  59  65  46  72  51  46  66]
 [103   0  81 107  38 110  55  94  76 123  88  76  69  86  99 113 108 106]
 [ 73  41   0  80  40  29  26  13  75  42  70  58  46  27  53  32  27  47]
 [ 28  48  47   0  55  59  52  60  62  58  43  31  40  21  59  48  53  61]
 [ 83  38  79  87   0 102  58  92  56 103  68  56  49  67  79  94 106  86]
 [ 76  44  72  83  43   0  98  16  46  64  73  61  73  67  27  40  30  43]
 [ 65  55  26  64  58  55   0  39  69  68  50  38  26  31  66  58  53  56]
 [ 60  28  56  67  27  16  82   0  62  48  75  63  69  51  40  24  14  34]
 [ 58  76  77  81  55  46  69  62   0  56  43  31  43  53  23  80  76  30]
 [ 74  79  42  84  71  35  68  51  56   0  59  47  54  37  59  24  65  43]
 [ 15  35  34  43  12  43  41  47  39  47   0  12  24  10  16  37  61  32]
 [ 27  47  46  50  24  5

Other variables:

In [22]:
max_path_length = n-(m-1)
paths = [[Int("p_%s_%s" % (i,j)) for j in range(max_path_length)] for i in range(m)] 
# create a matrix of mxn boolean variables for the assignment of items to couriers
assignment = [[Bool("a_%s_%s" % (i,j)) for j in range(n)] for i in range(m)]
# Create a solver instance
s = Solver()


Main constraints:

In [23]:
constraints = []
constraints.append(n>=m)

# Each item has a size > 0 and <= max(load size)
for i in range(n):
    constraints.append(And(item_size[i] > 0, item_size[i] <= max(load_size)))

# Each item is delivered at most by one courier, and at least by one courier
for i in range(n):
    constraints.append(Sum([If(paths[c][j] == i, 1, 0) for c in range(m) for j in range(max_path_length)]) == 1)

# assignement constraints 
for c in range(m):
    for i in range(n):
        constraints.append(assignment[c][i] == Or([paths[c][j] == i for j in range(max_path_length)]))

# Each courier can carry at most its load size
for c in range(m):
    constraints.append(Sum([If(assignment[c][i], item_size[i], 0) for i in range(n)]) <= load_size[c])

# Each courier must deliver at least one item
for c in range(m):
    constraints.append(Sum([If(assignment[c][i], 1, 0) for i in range(n)]) >= 1)

best_max_distance = math.inf
s.add(constraints)
courier_distances = [[0] for c in range(m)]
courier_loads = [[0] for c in range(m)]
best_courier_distances = [[0] for c in range(m)]
break_counter = 0
loop_counter = 0
for l in range(1000):
    if s.check() == sat:
        loop_counter += 1
        model = s.model()
        # get the values of the paths
        paths_values = [[model[paths[i][j]] for j in range(max_path_length)] for i in range(m)]
        # get path for each courier as a list of items, taking only the values in the range [0,n-1]
        paths_items = [[paths_values[i][j].as_long() for j in range(max_path_length) if paths_values[i][j].as_long() < n] for i in range(m)]
        paths_items = [list(filter(lambda x: x != -1, paths_items[i])) for i in range(m)] 
        # get the total distance for each courier by adding also the distance from the depot to the first item and from the last item to the depot
        for c in range(m):
            dist = distance[n][paths_items[c][0]] + distance[paths_items[c][-1]][n]
            for i in range(len(paths_items[c])-1):
                dist += distance[paths_items[c][i]][paths_items[c][i+1]]
            courier_distances[c][0] = dist

        # only accept solutions with max distance < best_max_distance
        # and update best_max_distance
        max_distance = max([courier_distances[c][0] for c in range(m)])
        if max_distance < best_max_distance:
            best_max_distance = max_distance
            break_counter = loop_counter
            best_courier_distances = courier_distances.copy()
            print("max_distance:", max_distance)
            print("paths_items:", paths_items)
            print("courier_distances:", courier_distances)
        else:
            if loop_counter - break_counter > 200:
                break


max_distance: 347
paths_items: [[1, 7, 14, 10], [3], [13, 8], [4, 16, 5, 6, 12, 11], [0], [9, 2, 15]]
courier_distances: [[273], [136], [142], [347], [123], [171]]
max_distance: 248
paths_items: [[8, 3], [13, 4, 14, 11], [10, 0, 15, 6], [16, 7, 9], [5], [2, 1, 12]]
courier_distances: [[172], [248], [222], [152], [86], [223]]
max_distance: 242
paths_items: [[11, 3], [12, 10, 9], [16, 0, 15, 8], [4, 14], [13, 5, 6], [7, 1, 2]]
courier_distances: [[141], [151], [231], [149], [242], [190]]
max_distance: 241
paths_items: [[0, 16, 1, 4], [3, 2, 5], [15, 10], [14, 7], [12, 11, 13], [9, 6, 8]]
courier_distances: [[241], [194], [150], [90], [121], [210]]
max_distance: 231
paths_items: [[9, 7, 14], [15, 3, 12], [6, 4, 13], [8, 11, 5], [10, 16, 2], [0, 1]]
courier_distances: [[150], [217], [231], [159], [192], [183]]
max_distance: 224
paths_items: [[16, 1], [15, 7, 5], [0], [12, 2, 3], [9, 8, 14], [10, 4, 13, 6, 11]]
courier_distances: [[167], [143], [123], [224], [138], [220]]
max_distance: 216


KeyboardInterrupt: 